In [1]:
import scanpy as sc
import scvi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from rich import print
#from scib_metrics.benchmark import Benchmarker
from scvi.model.utils import mde
from scipy.sparse import csr_matrix

/home/tbb/mambaforge-pypy3/envs/scvi/lib/python3.9/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/tbb/mambaforge-pypy3/envs/scvi/lib/python3.9/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/home/tbb/mambaforge-pypy3/envs/scvi/lib/python3.9/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/tbb/mambaforge-pypy3/envs/scvi/lib/python3.9/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed

In [2]:
scvi.__version__

'1.0.2'

In [3]:
adata_all = sc.read_h5ad("data/adata/all.h5ad")

In [4]:
adata_skin = sc.read_h5ad("data/adata/skin_scANVI.h5ad")

In [5]:
adata = adata_all[adata_skin.obs_names[adata_skin.obs["leiden_scANVI_02"].isin(["0", "2"])], :].copy()

/home/tbb/mambaforge-pypy3/envs/scvi/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [6]:
adata

AnnData object with n_obs × n_vars = 79021 × 36601
    obs: 'run', 'sum', 'detected', 'subsets_Mito_sum', 'subsets_Mito_detected', 'subsets_Mito_percent', 'total', 'sizeFactor', 'louvain', 'cluster', 'label', 'scDblFinder.class', 'scDblFinder.score', 'scDblFinder.weighted', 'scDblFinder.cxds_score', 'Doublet_cluster', 'doublet', 'dead_cell', 'cc_phase', 'cc_score_S', 'cc_score_G2M', 'ann_singleR_encode', 'cell_type_singleR_encode', 'ann_singleR_HCA', 'cell_type_singleR_HCA', 'cell_type', 'sample', 'tissue', 'study', 'donor', 'TRA.1.v_gene', 'TRA.1.d_gene', 'TRA.1.j_gene', 'TRA.1.c_gene', 'TRA.1.cdr3', 'TRA.1.cdr3_nt', 'TRA.1.umis', 'TRA.1.reads', 'TRA.1.high_confidence', 'TRA.1.productive', 'TRA.1.full_length', 'TRA.1.raw_clonotype_id', 'TRA.2.v_gene', 'TRA.2.d_gene', 'TRA.2.j_gene', 'TRA.2.c_gene', 'TRA.2.cdr3', 'TRA.2.cdr3_nt', 'TRA.2.umis', 'TRA.2.reads', 'TRA.2.high_confidence', 'TRA.2.productive', 'TRA.2.full_length', 'TRA.2.raw_clonotype_id', 'TRB.1.v_gene', 'TRB.1.d_gene', 'TRB.

In [7]:
adata.obs["cell_type"].value_counts()

cell_type
CD8+ T-cells         45263
CD4+ T-cells         31915
NK cells               865
Epithelial cells       327
Keratinocytes          236
Fibroblasts            127
DC                     126
B-cells                 56
Adipocytes              36
Endothelial cells       31
Monocyte                24
Erythrocytes             5
Melanocytes              4
Pre-B_cell_CD34-         3
Macrophage               2
HSC                      1
Name: count, dtype: int64

In [8]:
r = re.compile('^TR[ABGD][VDJ].*')
vmatch = np.vectorize(lambda x:bool(r.match(x)))

A = np.array(adata.var_names.values)
sel = vmatch(A)
A[sel]

array(['TRGJ2', 'TRGJP2', 'TRGJ1', 'TRGJP', 'TRGJP1', 'TRGV11', 'TRGVB',
       'TRGV10', 'TRGV9', 'TRGVA', 'TRGV8', 'TRGV7', 'TRGV6', 'TRGV5P',
       'TRGV5', 'TRGV4', 'TRGV3', 'TRGV2', 'TRGV1', 'TRBV1', 'TRBV2',
       'TRBV3-1', 'TRBV4-1', 'TRBV5-1', 'TRBV6-1', 'TRBV7-1', 'TRBV4-2',
       'TRBV6-2', 'TRBV7-2', 'TRBV8-1', 'TRBV5-2', 'TRBV6-4', 'TRBV7-3',
       'TRBV8-2', 'TRBV5-3', 'TRBV9', 'TRBV10-1', 'TRBV11-1', 'TRBV12-1',
       'TRBV10-2', 'TRBV11-2', 'TRBV12-2', 'TRBV6-5', 'TRBV7-4',
       'TRBV5-4', 'TRBV6-6', 'TRBV7-5', 'TRBV5-5', 'TRBV6-7', 'TRBV7-6',
       'TRBV5-6', 'TRBV6-8', 'TRBV7-7', 'TRBV5-7', 'TRBV7-9', 'TRBV13',
       'TRBV10-3', 'TRBV11-3', 'TRBV12-3', 'TRBV12-4', 'TRBV12-5',
       'TRBV14', 'TRBV15', 'TRBV16', 'TRBV17', 'TRBV18', 'TRBV19',
       'TRBV20-1', 'TRBV21-1', 'TRBV22-1', 'TRBV23-1', 'TRBV24-1',
       'TRBV25-1', 'TRBVA', 'TRBV26', 'TRBVB', 'TRBV27', 'TRBV28',
       'TRBV29-1', 'TRBD1', 'TRBJ1-1', 'TRBJ1-2', 'TRBJ1-3', 'TRBJ1-4',
       'TRBJ1-5

In [9]:
adata

AnnData object with n_obs × n_vars = 79021 × 36601
    obs: 'run', 'sum', 'detected', 'subsets_Mito_sum', 'subsets_Mito_detected', 'subsets_Mito_percent', 'total', 'sizeFactor', 'louvain', 'cluster', 'label', 'scDblFinder.class', 'scDblFinder.score', 'scDblFinder.weighted', 'scDblFinder.cxds_score', 'Doublet_cluster', 'doublet', 'dead_cell', 'cc_phase', 'cc_score_S', 'cc_score_G2M', 'ann_singleR_encode', 'cell_type_singleR_encode', 'ann_singleR_HCA', 'cell_type_singleR_HCA', 'cell_type', 'sample', 'tissue', 'study', 'donor', 'TRA.1.v_gene', 'TRA.1.d_gene', 'TRA.1.j_gene', 'TRA.1.c_gene', 'TRA.1.cdr3', 'TRA.1.cdr3_nt', 'TRA.1.umis', 'TRA.1.reads', 'TRA.1.high_confidence', 'TRA.1.productive', 'TRA.1.full_length', 'TRA.1.raw_clonotype_id', 'TRA.2.v_gene', 'TRA.2.d_gene', 'TRA.2.j_gene', 'TRA.2.c_gene', 'TRA.2.cdr3', 'TRA.2.cdr3_nt', 'TRA.2.umis', 'TRA.2.reads', 'TRA.2.high_confidence', 'TRA.2.productive', 'TRA.2.full_length', 'TRA.2.raw_clonotype_id', 'TRB.1.v_gene', 'TRB.1.d_gene', 'TRB.

In [10]:
adata = adata[:, sel == False].copy()
adata

/home/tbb/mambaforge-pypy3/envs/scvi/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


AnnData object with n_obs × n_vars = 79021 × 36373
    obs: 'run', 'sum', 'detected', 'subsets_Mito_sum', 'subsets_Mito_detected', 'subsets_Mito_percent', 'total', 'sizeFactor', 'louvain', 'cluster', 'label', 'scDblFinder.class', 'scDblFinder.score', 'scDblFinder.weighted', 'scDblFinder.cxds_score', 'Doublet_cluster', 'doublet', 'dead_cell', 'cc_phase', 'cc_score_S', 'cc_score_G2M', 'ann_singleR_encode', 'cell_type_singleR_encode', 'ann_singleR_HCA', 'cell_type_singleR_HCA', 'cell_type', 'sample', 'tissue', 'study', 'donor', 'TRA.1.v_gene', 'TRA.1.d_gene', 'TRA.1.j_gene', 'TRA.1.c_gene', 'TRA.1.cdr3', 'TRA.1.cdr3_nt', 'TRA.1.umis', 'TRA.1.reads', 'TRA.1.high_confidence', 'TRA.1.productive', 'TRA.1.full_length', 'TRA.1.raw_clonotype_id', 'TRA.2.v_gene', 'TRA.2.d_gene', 'TRA.2.j_gene', 'TRA.2.c_gene', 'TRA.2.cdr3', 'TRA.2.cdr3_nt', 'TRA.2.umis', 'TRA.2.reads', 'TRA.2.high_confidence', 'TRA.2.productive', 'TRA.2.full_length', 'TRA.2.raw_clonotype_id', 'TRB.1.v_gene', 'TRB.1.d_gene', 'TRB.

In [11]:
adata.X = csr_matrix(adata.X)
adata.layers["counts"] = adata.X.copy()

In [12]:
adata.obs["Malignant_exp"] = adata_skin.obs.Malignant[adata_skin.obs["leiden_scANVI_02"].isin(["0", "4"])]

In [13]:
del adata_all
del adata_skin

In [14]:
sc.pp.highly_variable_genes(
    adata,
    flavor="seurat_v3",
    n_top_genes=1000,
    layer="counts",
    batch_key="study",
    subset=True,
)

/home/tbb/mambaforge-pypy3/envs/scvi/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [15]:
scvi.model.SCVI.setup_anndata(
    adata, 
    layer="counts", 
    #categorical_covariate_keys=["run", "donor", "study"],
    categorical_covariate_keys=["run", "donor", "study", "chemistry"],
    continuous_covariate_keys=["cc_score_S", "cc_score_G2M"]
)

In [16]:
vae = scvi.model.SCVI(adata, n_layers=2, n_latent=20, gene_likelihood="nb")

In [ ]:
vae.train()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 89/101:  87%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                 | 88/101 [10:20<01:35,  7.38s/it, v_num=1, train_loss_step=268, train_loss_epoch=255]

In [ ]:
adata.obsm["X_scVI"] = vae.get_latent_representation()

In [ ]:
sc.pp.neighbors(adata, use_rep="X_scVI")
sc.tl.umap(adata, min_dist=0.3)
adata.obsm["X_umap_scVI"] = adata.obsm["X_umap"]

In [ ]:
perm_inds = np.random.permutation(len(adata))
sc.pl.embedding(
    adata[perm_inds],
    basis="X_umap_scVI",
    color=["run", "donor", "Malignant", "Malignant_exp"],
    frameon=False,
    colorbar_loc="None",
    ncols=1,
)

In [ ]:
lvae = scvi.model.SCANVI.from_scvi_model(
    vae,
    adata=adata,
    labels_key="Malignant",
    unlabeled_category="Unknown",
)

In [ ]:
lvae.train(max_epochs=50, n_samples_per_label=100)

In [ ]:
adata.obsm["X_scANVI"] = lvae.get_latent_representation(adata)

In [ ]:
sc.pp.neighbors(adata, use_rep="X_scANVI")
sc.tl.leiden(adata)
adata.obs["leiden_scANVI"] = adata.obs["leiden"]

In [ ]:
#adata.obsm["X_mde_scanvi"] = mde(adata.obsm["X_scANVI"])
sc.tl.umap(adata, min_dist=0.3)
adata.obsm["X_umap_scANVI_03"] = adata.obsm["X_umap"]

In [ ]:
sc.tl.umap(adata, min_dist=0.2)
adata.obsm["X_umap_scANVI_02"] = adata.obsm["X_umap"]
adata.obsm["X_umap_scANVI"] = adata.obsm["X_umap_scANVI_02"]

In [ ]:
sc.tl.umap(adata, min_dist=0.1)
adata.obsm["X_umap_scANVI_01"] = adata.obsm["X_umap"]
adata.obsm["X_umap_scANVI"] = adata.obsm["X_umap_scANVI_01"]

In [ ]:
perm_inds = np.random.permutation(len(adata))
sc.pl.embedding(
    adata[perm_inds],
    basis="X_umap_scANVI",
    color=["run", "donor", "leiden", "Malignant"],
    frameon=False,
    colorbar_loc="None",
    ncols=1,
)

In [ ]:
perm_inds = np.random.permutation(len(adata))
sc.pl.embedding(
    adata[perm_inds],
    basis="X_umap_scANVI",
    color=["study", "chemistry"],
    frameon=False,
    colorbar_loc="None",
    ncols=1,
)

In [ ]:
sc.tl.leiden(adata, key_added="leiden_scANVI_02", resolution=0.2)
sc.tl.leiden(adata, key_added="leiden_scANVI_03", resolution=0.3)
sc.tl.leiden(adata, key_added="leiden_scANVI_04", resolution=0.4)
sc.tl.leiden(adata, key_added="leiden_scANVI_05", resolution=0.5)
sc.tl.leiden(adata, key_added="leiden_scANVI_07", resolution=0.7)
sc.tl.leiden(adata, key_added="leiden_scANVI_10", resolution=1.0)
sc.tl.leiden(adata, key_added="leiden_scANVI_15", resolution=1.5)
sc.tl.leiden(adata, key_added="leiden_scANVI_20", resolution=2.0)

In [ ]:
sc.pl.embedding(
    adata[perm_inds],
    basis="X_umap_scANVI",
    color=["leiden_scANVI_02", "leiden_scANVI_03", "leiden_scANVI_04", "leiden_scANVI_05", "leiden_scANVI_07", "leiden_scANVI_10", "leiden_scANVI_15", "leiden_scANVI_20"],
    legend_loc="on data",
    frameon=False,
    ncols=2,
)

In [ ]:
adata.layers["scvi_normalized"] = lvae.get_normalized_expression(library_size=10e4)

In [ ]:
adata.write_h5ad("data/adata/skin_Tcell_scANVI.h5ad")
vae.save("data/scVI/skin_Tcell_scVI", overwrite=True)
lvae.save("data/scVI/skin_Tcell_scANVI", overwrite=True)